In [8]:
import numpy as np
import matplotlib.pyplot as plt

Data Model: data = signal+noise    
signal           = Ax+mu  
x is a random number drawn from a standard normal distribution, < x >=0, <x^2>=1    
the noise is Gaussian with mean =< noise >=0 and variance = < ( x-< x >)**2 >=sigma^2

A and mu depend on the parameters a,b. a and b are the parameters we are interested in inferring.

In [9]:
#get A as a function of a,b
def get_A(a,b,dim=1):
    A = a**2*np.ones((dim,dim))+b/2.
    return A

In [10]:
#get mu as a function of a,b
def get_mu(a,b, dim=1):
    mu = a**3/np.exp(b)*np.ones(dim)
    return mu

Fiducial Parameters (this is the underlying truth)

In [11]:
a_0=2; b_0=0.5

In [12]:
A_0  = get_A(a_0,b_0)
mu_0 = get_mu(a_0, b_0)

Noise level

In [13]:
sigma = 0.1

In [195]:
def generate_noise(sigma,dim=1):
    #fill in
    return noise

In [196]:
def generate_signal(A,mu,sigma,rand_seed,dim=1):
    
    #fill in
    
    return data

In [197]:
def generate_data(A,mu,sigma,rand_seed=None,dim=1):
    
    #fill in
    
    return data

fiducial data (that's the assumed truth)

In [198]:
fid_data = generate_data(A_0,mu_0,sigma,rand_seed=123)

take derivatives at fixed random seed

Questions:  
1) What's the mean and the variance of the data? Which distribution does the data follow? (Write down an equation P(d)=...)   
2) How can I estimate the variance of the data, given a big enough data sample? Can you write some code for this? (maybe start with the mean) Does your estimate confirm your calculation from 1)?   
3) Write a finite differencing operatator with respect to a and b for the data.